In [2]:
from data_utils import *
from losses import *
net = load_model_from_checkpoint(
    'PATH_TO_CHECKPOINT')
device ='cuda'
net= net.to(device)

In [3]:
import torch.nn.functional as F
import os
import torch
import numpy as np
from sklearn.preprocessing import StandardScaler

directory = 'PATH_TO_FOLDER_WITH_EXP_DATA'
scaler = StandardScaler()

folders = [folder for folder in os.listdir(
    directory) if os.path.isdir(os.path.join(directory, folder))]

final_tensors = []
exp_tensors = []

for folder in folders:
    folder_path = os.path.join(directory, folder)

    pickle_files = [file for file in os.listdir(
        folder_path) if file.endswith('.pickle')]

    tensor_list = []

    count_large = 0

    for pickle_file in pickle_files:

        pickle_to_dict = np.load(os.path.join(
            folder_path, pickle_file), allow_pickle=True)

        df = make_numpy_array(pickle_to_dict)

        i, j = df.shape[0], df.shape[1]

        df = scaler.fit_transform(df.view(-1, 1))
        df = torch.tensor(df).view(i, j)
       
        tensor_list.append(df)

    tensored_stuff = torch.cat(tensor_list, dim=0)
    exp_tensors.append(tensored_stuff)
    

In [4]:
device= torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
output_embeddings=[]
for i in exp_tensors:
    i =i.transpose(1,0).unsqueeze(0).float().to(device)
    output_embeddings.append(net(i).detach().cpu().numpy())
        



In [5]:
output_embeddings=np.array(output_embeddings).squeeze()
output_embeddings.shape

(13, 4)

In [8]:
directory = 'PATH_TO_FOLDER_WITH_SIM_DATA'
scaler = StandardScaler()

folders = [folder for folder in os.listdir(
    directory) if os.path.isdir(os.path.join(directory, folder))]

final_tensors = []
exp_tensors_sims = []


for folder in folders:
    folder_path = os.path.join(directory, folder)
    #print(folder_path)

    pickle_files = [file for file in os.listdir(
        folder_path) if file.endswith('.pickle')]

    tensor_list = []

    count_large = 0

    for pickle_file in pickle_files:

        pickle_to_dict = np.load(os.path.join(
            folder_path, pickle_file), allow_pickle=True)

        df = make_numpy_array(pickle_to_dict)

        i, j = df.shape[0], df.shape[1]

        df = scaler.fit_transform(df.view(-1, 1))
        df = torch.tensor(df).view(i, j)
        #print(df.shape)
        tensor_list.append(df)

    tensored_stuff = torch.cat(tensor_list, dim=0)
    exp_tensors_sims.append(tensored_stuff)
    #
    # tensor_tensor_list.append(tensored_stuff)
    #arr=np.load('exp.npy', allow_pickle=True)


Checking the length distribution of Experiments

In [10]:
len_counter = []
for i in exp_tensors:
    length = i.shape[0]
    len_counter.append(length)

max_len = max(len_counter)
print(max_len)
print(np.array(len_counter).mean())


8733
3893.5384615384614


Checking the length distribution of Simulations

In [11]:
len_counter = []
for i in exp_tensors_sims:
    length = i.shape[0]
    len_counter.append(length)

max_len = max(len_counter)
print(max_len)
print(np.array(len_counter).mean())

1377
1198.0


In [12]:

output_embeddings_sims = []
for i in exp_tensors_sims:
    i = i.transpose(1, 0).unsqueeze(0).float().to(device)
    output_embeddings_sims.append(net(i).detach().cpu().numpy())

output_embeddings_sims = np.array(output_embeddings_sims).squeeze()
output_embeddings_sims.shape

# Ensure that a particular order is maintained for exps and sims

order =[12,6,3,0,2,8,11,7,10,4,5,9,1]
output_embeddings_sims = output_embeddings_sims[order]



Cosine Similarity

In [13]:
similarity_scores = []
for i in range(13):
    sim = torch.tensor(output_embeddings[i]) 
    exp = torch.tensor(output_embeddings_sims[i])
    similarity = torch.tensor(F.cosine_similarity(
        sim.unsqueeze(0), exp.unsqueeze(0), dim=1))


    similarity_scores.append(similarity.item())

np.mean(similarity_scores)


/tmp/ipykernel_45183/716125333.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  similarity = torch.tensor(F.cosine_similarity(


0.9479733201173636

Scaling Experiments And Simulations (Embeddings)

In [14]:
scaler=StandardScaler()
scaled_embeds_exp=[]
scaled_embeds_sims=[]
for i in output_embeddings:
    j=scaler.fit_transform(i.reshape(-1,1))
    scaled_embeds_exp.append(j)

for i in output_embeddings_sims:
    j=scaler.fit_transform(i.reshape(-1,1))
    scaled_embeds_sims.append(j)
    

Euclidean Similarity for normalized inputs

In [15]:
similarity_scores = []
for i in range(13):
    sim = torch.tensor(scaled_embeds_exp[i]) 
    exp = torch.tensor(scaled_embeds_sims[i])

    euclidean_distance = torch.norm(
        sim - exp, p=2)  

    similarity_scores.append(similarity.item())
    similarity = 1/(1+euclidean_distance)

np.mean(similarity_scores)


0.6612705680040213